In [ ]:
# Basic imports

import sys

import numpy as np

import xarray as xr

import matplotlib.pyplot as plt

from IPython.display import HTML

import datetime
from cftime import num2date

import cmocean  # for nice oceanographic colourmaps

#JUPYTER notebook magics
%matplotlib inline 

In [ ]:
%%bash

# We tune the forcing using these two modules for the atmosphere and ocean boundaries
/home/nextsimdg/build/nextsim --help-config FluxConfiguredAtmosphere
/home/nextsimdg/build/nextsim --help-config FluxConfiguredOcean

In [ ]:
# Create initial conditions using the make_init_base.py class found in nextsimdg/run

sys.path.append('../../nextsimdg/run')

from make_init_base import initMaker

# Creates initial conditions for the Bjornsson et al. (2001) polynya case

# Domain size [km]
x = 100
y = 50
res = 2

nfirst = int(y / res)
nsecond = int(x / res)
nLayers = 3

# The model expects everything in metres
initializer = initMaker("init_polynya.nc", nfirst, nsecond, nLayers, res*1e3)

# Ice everywhere and all boundaries closed, except the x = 100 km end
initializer.mask[:, :] = 1.
initializer.mask[0, :] = 0.
initializer.mask[-1, :] = 0.
initializer.mask[:, 0] = 0.
#initializer.mask[:, -1] = 0. ## right

# Uniform concentration of 90%
initializer.cice[:, :] = 0.9

# Uniform thickness of 20 cm
initializer.hice[:, :] = 0.2

# Undamaged ice
initializer.damage[:, :] = 1.

# Ice and ocean temperature and salinity at the freezing point
ice_salinity = 5  # should match Ice::s in constants.hpp
mu: float = -0.055  # should match Water::mu in constants.hpp
ocean_temperature = -1.54
ocean_salinity = ocean_temperature / mu

initializer.sss[:, :] = ocean_salinity
initializer.sst[:, :] = ocean_temperature
initializer.tice[:, :, :] = ice_salinity * mu

"""
In a normal script, the file is written when initializer goes out of scope - but in 
Jupyter, we need to call __writeFile__ explicitly
"""
initializer.__writeFile__()

In [ ]:
%%bash

# Run the  model with the polynya config file
time /home/nextsimdg/build/nextsim --config-file config_files/config_polynya.cfg

In [ ]:
# Load the NetCDF file
ds = xr.open_dataset("polynya.diagnostic.nc", group="/data")
print(ds)

# We (still) need to fetch the mask from the init file

mask = xr.open_dataset("init_polynya.nc", group="/data")['mask']
land = np.where(mask == 0, 1, np.nan)  # 1 = land, nan = ocean

In [ ]:
# Create sensible dates to use and a land mask
time = ds['time']
time_vals = num2date(time.values, units='seconds since 1970-01-01', calendar='gregorian')

time_index = 5

In [ ]:
# Striding for u and v
ny, nx = ds['u'].isel(time=0).shape
Y, X = np.meshgrid(np.arange(ny), np.arange(nx), indexing='ij')

stride = 2
u = ds['u'].isel(time=time_index)[::stride, ::stride]
v = ds['v'].isel(time=time_index)[::stride, ::stride]
x = X[::stride, ::stride]
y = Y[::stride, ::stride]

In [ ]:
# NB - we need to pick the first DG component, as well as a time slice
var = ds['cice'].isel(time=time_index).isel(dg_comp=0)

plt.figure()
p = plt.pcolormesh(var, shading='auto', cmap='cmo.ice', vmin=0, vmax=1)
plt.gca().set_aspect('equal')

plt.colorbar(p, label='Sea Ice Concentration', orientation='horizontal')

plt.quiver(x, y, u, v, scale=5, color='g', width=0.002)

plt.pcolormesh(land, shading='auto', cmap='Pastel2', vmin=0, vmax=1)

plt.title(f"Polynya model at {time_vals[time_index].isoformat()}")

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
var = ds['hice'].isel(time=time_index).isel(dg_comp=0)

plt.figure()
p = plt.pcolormesh(var, shading='auto', cmap='viridis', vmin=0, vmax=0.3)
plt.gca().set_aspect('equal')

plt.colorbar(p, label='Sea Ice Thickness', orientation='horizontal')

plt.quiver(x, y, u, v, scale=5, color='w', width=0.002)

plt.pcolormesh(land, shading='auto', cmap='Pastel2', vmin=0, vmax=1)

plt.title(f"Polynya model at {time_vals[time_index].isoformat()}")

plt.xticks([])
plt.yticks([])
plt.show()

In [ ]:
ds.close()